# Lab 3 Solving an optimisation problem (TSP) using three local search algorithms
This notebook includes four parts:
1. Problem definition
2. Implementation of Hill Climbing (HC) algorithm and its application to solve the target optimisation problem
3. Implementation of Genetic Algorithm (GA) and its application to solve the target optimisation problem
4. Implementation of Simulated Annealing (SA) algorithm and its application to solve the target optimisation problem

## Part 1 -- Problem definition
### Search problem: Travelling salesman problem (TSP)

In the TSP, an agent must start from a given city, visit every other city exactly once, and return to the start city, while minimizing travel cost. In this example, we consider a region, say Romania, with known city locations.

To define a TSP problem, we need to have the following elements:
1. All cities with their names and locations.
2. Connections between cities, including the distances (weights) between each pair.
3. A neighbour function to generate the neighbouring solution for a given solution.
4. A cost or objective function to assess the quality of a solution."

Import a Python data analysis library, pandas, and give it a short name 'pd'

In [1]:
import pandas as pd

Use `read_csv()` method in `pd` to read a csv file, `problemData.csv`, which contains cities and their locations and save the output into a dataframe, called `df`. Note. The csv file should be in the current directory.

In [2]:
df = pd.read_csv('problemData.csv')

In [3]:
df.head()

,LocationName,LocationX,LocationY
0,Arad,91,492
1,Bucharest,400,327
2,Craiova,253,288
3,Drobeta,165,299
4,Eforie,562,293


Create a dictionary, called `locations`, to store the city names (keys) and their respective locations (values).
One element of the dictionary should be like `"Arad": (91, 492)`.

A useful Python function `zip()` is used.
"The zip() function in Python is a built-in utility that combines two or more iterables (like lists, tuples, or strings) element-wise into tuples. Each tuple contains the elements from the corresponding positions of the input iterables. The resulting tuples are collected into an iterator, which can be converted into other data types like lists or tuples." *from ChatGPT 40 mini*

In [4]:
locations = {}
for x1,x2 in zip(df["LocationName"],zip(df["LocationX"],df["LocationY"])):
    locations[x1] = x2

In [5]:
locations

{'Arad': (91, 492),
 'Bucharest': (400, 327),
 'Craiova': (253, 288),
 'Drobeta': (165, 299),
 'Eforie': (562, 293),
 'Fagaras': (305, 449),
 'Giurgiu': (375, 270),
 'Hirsova': (534, 350),
 'Iasi': (473, 506),
 'Lugoj': (165, 379),
 'Mehadia': (168, 339),
 'Neamt': (406, 537),
 'Oradea': (131, 571),
 'Pitesti': (320, 368),
 'Rimnicu': (233, 410),
 'Sibiu': (207, 457),
 'Timisoara': (94, 410),
 'Urziceni': (456, 350),
 'Vaslui': (509, 444),
 'Zerind': (108, 531)}

In [6]:
import numpy as np

def define_map(city_location):
    global distances
    city_locations = city_location
    all_cities = []
    distances = {}

    for city in city_locations.keys():
        distances[city] = {}
        all_cities.append(city)
    all_cities.sort()

    for name_1, coordinates_1 in city_locations.items():
            for name_2, coordinates_2 in city_locations.items():
                distances[name_1][name_2] = np.linalg.norm(
                    [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
                distances[name_2][name_1] = distances[name_1][name_2]

    return (all_cities, distances)

Call the `define_map()` method using the previously generated dictionary `locations` and save the two outputs in `all_cities` and `distances`, respectively.

In [7]:
all_cities, distances = define_map(locations)

Display the top k elements of the nested dictionary `distances` obtained from calling the `define_map()` method. k is obtained from a user.

In [ ]:
from itertools import islice

num = int(input ("input K as the number of items in a dictionary to display: "))
top_K_items = dict(islice(distances.items(), num))
top_K_items

Define a function `generate_k_neighbouring_paths(path, k)` that generates k neighbouring paths for a given path for the Traveling Salesman Problem. This fucntion is the transition model that is specific to TSP.

A path is a sequence of cities to be visited, without having the start city at the end. k is an integer for the number of neighbouring paths created for a given path, default value is 2.

Follow the pseudo code below to define this function:

Function generate_k_neighbouring_paths(path, k):

    Initialize current_path as a copy of path
    Initialize neighbouring_path as a copy of path
    Initialize an empty list neighbouring_paths

    For i from 1 to k:
        Generate a random integer 'left' between 1 and (length of current_path- 1)
        Generate a random integer 'right' between 1 and (length of current_path - 1)
        If left is greater than right:
            Swap left and right
        Reverse the segment of neighbouring_path from index left to index right (inclusive)
        Append neighbouring_path to neighbouring_paths
        Reset neighbouring_path to a copy of path
    
    Return neighbouring_paths


In [ ]:
import random

def generate_k_neighbouring_paths(path, k=2):
    # Answer
    current_path = path[:]
    neighbouring_path = path[:]
    neighbouring_paths = []
    for i in range(k):
        left = random.randint(1, len(current_path) - 1)
        right = random.randint(1, len(current_path) - 1)
        if left > right:
            left, right = right, left
        neighbouring_path[left: right + 1] = reversed(neighbouring_path[left: right + 1])
        neighbouring_paths.append(neighbouring_path)
        neighbouring_path = path[:]
    return neighbouring_paths

Define a cost function, `cost_fun(path)`, that takes a path as input and returns its total distance as the cost.

In various optimisation problems, this is often referred to as an evaluation function, a fitness function, or an objective function. This function assesses the quality of a given solution in the context of the problem at hand.

In the context of the Travelling Salesman Problem, the 'cost' of a path is the total distance that the salesman would travel by following that path. This includes the distance from the last city in the path back to the start city.

A lower cost indicates a shorter total distance and thus a better solution to the problem.

* The input parameter: `path` is a list of cities representing the order in which the cities are visited.

* The returned value: `total_distance` is the total distance of the path as a float number, including the return trip from the last city in the list to the start city.

Use the nested dictionary 'distances', which is created earlier for storing the distances between each pair of cities, to extrat the distance between two cities for calculating the path cost.

Follow the pseudo code below to define this function:

Function cost_fun(path):

    Initialize total_distance as 0

    For i from 0 to (length of path - 2):
        Set city1 as path[i]
        Set city2 as path[i + 1]
        Add distance between city1 and city2 from distances to total_distance
    
    Set start_city as path[0]
    Set final_city as path[length of path - 1]
    Add distance between start_city and final_city from distances to total_distance

    Return total_distance

In [ ]:
def cost_fun(path):
    # Initialize total distance to 0
    total_distance = 0

    # Calculate the distance for each pair of consecutive cities in the path
    for i in range(len(path) - 1):
        city1 = path[i]
        city2 = path[i + 1]
        total_distance += distances[city1][city2]

    # Add the distance from the last city back to the first city
    start_city = path[0]
    final_city = path[-1]
    total_distance += distances[start_city][final_city]

    return total_distance


Specify the start city, called `start_city`

In [ ]:
start_city = 'Arad'

Enter the start city: Arad


Here is the end of Part 1 -- problem defition. You need to learn how to define an optimisation problem from this sample problem defition!

## Part 3 -- Implementation of Genetic Algorithm (GA) and its application to solve the target optimisation problem

In [ ]:
from collections import Counter

def test_duplicated(x):
    duplicated = False
    duplicate = dict(Counter(x))
    for key,value in duplicate.items():
        if value > 1:
            duplicated = True
    return duplicated

In [ ]:
import random

def generate_for_Non_Duplication(x, y):
    new_gen1 = x[:]
    new_gen2 = y[:]
    new_gen1.remove(start_point)
    new_gen2.remove(start_point)
    n = len(new_gen1)
    new_gen = []
    while True:
        c = random.randint(0, n)
        new_gen = new_gen1[:c] + new_gen2[c:]
        if test_duplicated(new_gen):
            pass
        else:
            new_gen.insert(0, start_point)
            break
    return new_gen

In [ ]:
import random

def mutate_for_Non_Duplication(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    mutate_population = x[:]
    mutate_population.remove(start_point)
    random.shuffle(mutate_population)
    mutate_population.insert(0, start_point)
    return mutate_population

In [ ]:
import random

def recombine_for_Non_Duplication(x, y):
    new_gen1 = x[:]
    new_gen2 = y[:]
    new_gen1.remove(start_point)
    new_gen2.remove(start_point)
    n = len(new_gen1)
    duplicated = False
    c = random.randrange(0, n)
    new_gen = new_gen1[:c] + new_gen2[c:]
    new_gen.insert(0, start_point)
    duplicated = test_duplicated(new_gen)
    if duplicated:
        new_gen = generate_for_Non_Duplication(x, y)
    return new_gen

Define a a random-sample function that picks from seq weighted by weights, which is used in definition of the select operator, `select()`

In [ ]:
import bisect
def weighted_sampler(seq, weights):

    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]

In [ ]:
def select(population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(2)]

Define a method `fitness_threshold()` to return a better individual than a given threshold.

In [ ]:
def fitness_threshold(fitness_fn, f_thres, population):
    if not f_thres:
        return None

    fittest_individual = max(population, key=fitness_fn)
    if fitness_fn(fittest_individual) >= f_thres:
        return fittest_individual

    return None


In [ ]:
import random

def recombine(x, y):
    n = len(x)
    c = random.randrange(0, n)
    return x[:c] + y[c:]

In [ ]:
import random

def mutate(x, gene_pool, pmut):
    if random.uniform(0, 1) >= pmut:
        return x

    n = len(x)
    g = len(gene_pool)
    c = random.randrange(0, n)
    r = random.randrange(0, g)

    new_gene = gene_pool[r]
    return x[:c] + [new_gene] + x[c + 1:]

In [ ]:
def init_population_for_Non_Duplication(pop_number, gene_pool, state_length, start):
    g = len(gene_pool)
    initial_population = gene_pool.copy()
    population = []
    for i in range(pop_number):
        random.shuffle(initial_population)
        initial_population.remove(start)
        initial_population.insert(0,start)
        population.append(initial_population)
        initial_population = gene_pool.copy()
    return population



In [ ]:
def genetic_algorithm(population, fitness_fn, gene_pool, f_thres=None, ngen=None, pmut=0.1, start = None, non_duplication = True):
    global start_point
    start_point = start
    new_population=[]
    if non_duplication:
        for generation in range(ngen):
            for i in range(len(population)):
               individual1, individual2 = select(population, fitness_fn);
               individual_recombined = recombine_for_Non_Duplication(individual1, individual2)
               individual_final = mutate_for_Non_Duplication(individual_recombined, pmut)
               new_population.append(individual_final)

            population = new_population

           # population = [mutate_for_Non_Duplication(recombine_for_Non_Duplication(*select(population, fitness_fn)), gene_pool, pmut) for i in range(len(population))]

            # stores the fittest individual genome in the current population
            current_best = max(population, key = fitness_fn)
            print("Generation: {}\t\tFitness: {}\r".format(str(generation),fitness_fn(current_best)))

            # compare the fitness of the current best individual to f_thres
            fittest_individual = fitness_threshold(fitness_fn, f_thres, population)

            # if fitness is greater than or equal to f_thres, we terminate the algorithm
            if fittest_individual:
                return fittest_individual, generation
    else:
        for generation in range(ngen):
            population = [mutate(recombine(*select(population, fitness_fn)), gene_pool, pmut) for i in range(len(population))]
            # stores the individual genome with the highest fitness in the current population
            current_best = max(population, key = fitness_fn)
            print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')

            # compare the fitness of the current best individual to f_thres
            fittest_individual = fitness_threshold(fitness_fn, f_thres, population)
            print(fittest_individual)

            # if fitness of the best individual is greater than or equal to f_thres, we terminate the algorithm
            if fittest_individual:
                return fittest_individual, generation

    return max(population, key=fitness_fn) , generation # for using a fitness function, the smaller value, the better


Define the fitness function, `fitness_fun()`, as the difference between a big positive number, called `cap`, and the actual cost, which is the path cost using `cost_fun(path)`. The big number can be close infinit to make sure the fitness function has positive values.

In [ ]:
cap = float(input ("Provide the cap value for making fitness function produce positive values. It should be a very big number. Suggested value is 5000"))
def fitness_fun(path):
    return (cap - cost_fun(path))

Provide the cap value for making fitness function produce positive values. It should be a very big number. Suggested value is 50005000


Create a gene pool using all the cities (`all_cities`), called `gene_pool`, which will be used to form solutions.

In [ ]:
gene_pool = all_cities

Specify the maximum population size, called `max_population`. This parameter is problem specific. In this example, we set it to be 100.

In [ ]:
max_population = 100

Use the method `init_population_for_Non_Duplication(size-population, gene-pool, number-of-elements-in-gene-pool, start_city)`, to generate the intial population for the given maximum population size, called `population`.

In [ ]:
population = init_population_for_Non_Duplication(max_population, gene_pool, len(gene_pool), start_city)

In [ ]:
print(len(population))

10


Define the mutation rate (`mutation_rate`). This parameter is typically quite small. In this case, we'll use 0.05 to speed up the search process.

Note. This parameter is problem specific.

In [ ]:
mutation_rate = 0.05

Define the termination criteria to control when the algorithm should stop.

In this case, we will use two criteria:
* the maximum number of generations (`max_gen`), say 1000, and
* the threshold of the fitness function (`f_thres`), say 4000.

The algorithm will stop when either the maximum number of generations is reached or the fitness function value exceeds the threshold.

In [ ]:
max_gen = 1000
f_thres = 4000

Apply the Genetic Algorithm to solve the problem.

Call the function using `genetic_algorithm(population, fitness-function, gene-pool, fitness-function-threshold, max-number-generation-to-run, mutation-rate, start-point)`and save the outputs in two variables, `solution` and `generations`, respectively.

The function will also display the search results at each step, including the generation number and the corresponding fitness function value.

In [ ]:
solution, generations = genetic_algorithm(population, fitness_fun, gene_pool, f_thres, max_gen, mutation_rate, start_city)

Generation: 0		Fitness: 902.4145060846176
Generation: 1		Fitness: 902.4145060846176
Generation: 2		Fitness: 902.4145060846176
Generation: 3		Fitness: 902.4145060846176
Generation: 4		Fitness: 1311.0789796924055
Generation: 5		Fitness: 1311.0789796924055
Generation: 6		Fitness: 1311.0789796924055
Generation: 7		Fitness: 1311.0789796924055
Generation: 8		Fitness: 1460.016149132236
Generation: 9		Fitness: 1460.016149132236


In [ ]:
print("The best solution is: \n{} and \nThe cost is: \n{}".format(solution, cost_fun(solution)))

The best solution is: 
['Arad', 'Neamt', 'Hirsova', 'Urziceni', 'Rimnicu', 'Iasi', 'Fagaras', 'Craiova', 'Giurgiu', 'Pitesti', 'Lugoj', 'Bucharest', 'Vaslui', 'Eforie', 'Drobeta', 'Sibiu', 'Oradea', 'Zerind', 'Mehadia', 'Timisoara'] and 
The cost is: 
3539.983850867764


Display the best solution from the initial population and its corresponding cost for comparison with the final result.

In [ ]:
def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items

In [ ]:
def argmin_random_tie(seq, key):
    """Return a minimum element of seq; break ties at random."""
    return min(shuffled(seq), key=key)

In [ ]:
evaluation_fn = lambda x: cost_fun(x)
init_representation = argmin_random_tie(population, key= evaluation_fn)
print("The initial solution is:\n{}, and \nThe cost is: \n{}".format(init_representation, cost_fun(init_representation)))

The initial solution is:
['Arad', 'Oradea', 'Iasi', 'Sibiu', 'Eforie', 'Giurgiu', 'Urziceni', 'Hirsova', 'Craiova', 'Lugoj', 'Fagaras', 'Mehadia', 'Drobeta', 'Zerind', 'Neamt', 'Timisoara', 'Pitesti', 'Bucharest', 'Rimnicu', 'Vaslui', 'Arad', 'Arad', 'Arad'], and 
The cost is: 
4344.03830914048
